In [2]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm

sys.path.append('../')

from torch_geometric.data import Data
from torch_geometric.contrib.nn.models import GRBCDAttack
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from engine.model.GCN import SimpleGCN

from torch_geometric.utils import to_networkx

from torch_geometric.utils import degree
from networkx import core_number, eigenvector_centrality, harmonic_centrality, katz_centrality, closeness_centrality, betweenness_centrality

Data load

In [5]:
dataName = 'Cora'

dataset = Planetoid(f'./{dataName}', dataName, pre_transform=NormalizeFeatures())[0]
dataset = Data(x = dataset.x, edge_index = dataset.edge_index) 
dataset = to_networkx(dataset, node_attrs = ["x"], to_undirected= False)

Feature calculate

In [ ]:

coreNum = core_number(dataset)

budget load

In [ ]:
import pickle
with open(f'../result/pickles/{dataName}Budget.pkl', 'rb') as f:
    budget = pickle.load(f)

In [ ]:
pfeat = []
pbud = []
budList = []
dist = []
mv = 0
for b in budget:
    if b not in budList:
        budList.append(b)
budList.sort()
dist = [[] for d in budList]
for i, b in enumerate(budget):
    if b > 0:
        pbud.append(b)
        pfeat.append(coreNum[i])
        dist[budList.index(b)].append(coreNum[i])

plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for i, d in enumerate(dist):
    for v in d:
        plt.scatter(v,budList[i], c = 'b', alpha = 1 / len(d))
title = f'{dataName} core and budget'
plt.title(title)
plt.savefig(f'../result/plot/{title}.png')
plt.show()

covariance

In [ ]:
print(max(pbud), max(pfeat))
ppbud = [x/max(pbud) for x in pbud]
ppfeat = [x/max(pfeat) for x in pfeat]

covval = np.cov(ppfeat, ppbud)[0,1]
print(covval)